Допустим, у нас есть $n$ товаров с заданными стоимостями $v_i$ и массой $w_i$. В сумку убирается $С$ кг. Сколько какого товара взять, чтобы сумма всех стоимостей товаров была наибольшей?

In [2]:
values = [4, 20, 1, 5, 3, 10]
weights = [5, 10, 2, 5, 6, 6]
C = 19
n = 6

Сформулируем задачу:
$$\max\sum v_i x_i$$
$$\sum w_i x_i \leq C $$

Как должна выглядеть задача:
$$\min c^T x$$
$$A x \leq b $$

Получается, что $c=-v$, $A=w^T$, $b=(C)$

In [3]:
import numpy as np

In [4]:
c = - np.array(values)
A = np.array(weights)         #shape = (6,)
A = np.expand_dims(A, 0)      #shape = (1,6)
b = np.array([C])

In [5]:
from scipy.optimize import linprog

In [6]:
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: -38.000000003742656
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([-3.30540573e-09])
  status: 0
 success: True
       x: array([1.42705574e-11, 1.90000000e+00, 3.55393212e-11, 3.84018663e-11,
       2.12247194e-10, 2.86840312e-10])

In [1]:
!pip install cvxpy

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\shjb\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\shjb\\AppData\\Local\\Temp\\pip-install-il1yx1fg\\ecos\\setup.py'"'"'; __file__='"'"'C:\\Users\\shjb\\AppData\\Local\\Temp\\pip-install-il1yx1fg\\ecos\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\shjb\AppData\Local\Temp\pip-wheel-ugcwtyz_'
       cwd: C:\Users\shjb\AppData\Local\Temp\pip-install-il1yx1fg\ecos\
  Complete output (12 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\ecos
  copying src\ecos\ecos.py -> build\lib.win-amd64-3.8\ecos
  copying src\ecos\version.py -> build\lib.win-amd64-3.8\ecos
  copying src\ecos\__init__.py -> build\lib.win-amd64-3.8\ecos



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Running setup.py clean for ecos
Failed to build ecos scs



  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\scs
  copying src\__init__.py -> build\lib.win-amd64-3.8\scs
  running build_ext
  blas_mkl_info:
    NOT AVAILABLE
  blis_info:
    NOT AVAILABLE
  openblas_info:
      library_dirs = ['C:\\projects\\numpy-wheels\\numpy\\build\\openblas']
      libraries = ['openblas']
      language = f77
      define_macros = [('HAVE_CBLAS', None)]
  blas_opt_info:
      library_dirs = ['C:\\projects\\numpy-wheels\\numpy\\build\\openblas']
      libraries = ['openblas']
      language = f77
      define_macros = [('HAVE_CBLAS', None)]
  lapack_mkl_info:
    NOT AVAILABLE
  openblas_lapack_info:
      library_dirs = ['C:\\projects\\numpy-wheels\\numpy\\build\\openblas']
      libraries = ['openblas']
      language = f77
      define_macros = [('HAVE_CBLAS', None)]
  lapack_opt_info:
      library_dirs = ['C:\\projects\\numpy-wheels\\numpy\\build\\openblas']
      libraries = ['openblas']
      

In [21]:
import cvxpy

In [22]:
x = cvxpy.Variable(shape=n, integer = True)

In [23]:
constraint = (A @ x <= b)
total_value = c * x

C:\Users\SVasilchenko\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:556: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 4 times so far.

  warnings.warn(msg, UserWarning)


In [24]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

In [25]:
problem.solve(solver='ECOS_BB')

-159383570.99999994

In [26]:
x.value

array([-8388608.,  8388608., -8388608., -5033161., -8388608.,  8388608.])

Теперь положительные $x$

In [27]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve(solver='ECOS_BB')

C:\Users\SVasilchenko\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:556: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 5 times so far.

  warnings.warn(msg, UserWarning)


-31.000000028568287

In [28]:
x.value.round(0)

array([0., 1., 1., 0., 0., 1.])

Теперь $x = 0$ или $1$

In [29]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = A @ x <= b
x_positive = x >= 0
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve(solver='ECOS_BB')

C:\Users\SVasilchenko\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:556: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 6 times so far.

  warnings.warn(msg, UserWarning)


-31.000000028568287

In [30]:
x.value.round(0)

array([0., 1., 1., 0., 0., 1.])